In [64]:
import numpy as np
import data
from sklearn.decomposition import PCA
from sklearn.neighbors import LocalOutlierFactor as LOF
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
import nn
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
train_data = data.Dataset('train.csv')
#train_data.to_one_hot()

In [62]:
lof = LOF(n_neighbors=5)
outlier_labels = lof.fit_predict(train_data.x)
train_data.x = train_data.x[outlier_labels == 1]
train_data.y = train_data.y[outlier_labels == 1]

In [54]:
scaler = StandardScaler()
train_data.x = scaler.fit_transform(train_data.x)

In [63]:
pca = PCA(n_components=256)
pca.fit(train_data.x)
train_data.x = pca.transform(train_data.x)
np.sum(pca.explained_variance_ratio_)


0.9841102719104073

In [56]:
train_x,test_x,train_y,test_y = train_test_split(train_data.x,train_data.y,test_size=0.1)

In [57]:
print(train_x.shape,train_y.shape)

(1084, 256) (1084,)


In [60]:
nn.train(train_data.x,train_data.y,validation_split=0.1,epochs=200,weight_decay=1.5*10e-6)

Epoch [1/200], Loss: 2.9582
Epoch [2/200], Loss: 2.9070
Epoch [3/200], Loss: 2.8269
Epoch [4/200], Loss: 2.7325
Epoch [5/200], Loss: 2.6879
Epoch [6/200], Loss: 2.5820
Epoch [7/200], Loss: 2.5175
Epoch [8/200], Loss: 2.5303
Epoch [9/200], Loss: 2.4889
Epoch [10/200], Loss: 2.4579
Epoch [11/200], Loss: 2.3553
Epoch [12/200], Loss: 2.2711
Epoch [13/200], Loss: 2.2685
Epoch [14/200], Loss: 2.2977
Epoch [15/200], Loss: 2.3081
Epoch [16/200], Loss: 2.2920
Epoch [17/200], Loss: 2.2687
Epoch [18/200], Loss: 2.2678
Epoch [19/200], Loss: 2.2859
Epoch [20/200], Loss: 2.2666
Epoch [21/200], Loss: 2.1990
Epoch [22/200], Loss: 2.2009
Epoch [23/200], Loss: 2.2452
Epoch [24/200], Loss: 2.1426
Epoch [25/200], Loss: 2.1557
Epoch [26/200], Loss: 2.2157
Epoch [27/200], Loss: 2.1611
Epoch [28/200], Loss: 2.1431
Epoch [29/200], Loss: 2.1851
Epoch [30/200], Loss: 2.1279
Epoch [31/200], Loss: 2.1374
Epoch [32/200], Loss: 2.1802
Epoch [33/200], Loss: 2.1710
Epoch [34/200], Loss: 2.1093
Epoch [35/200], Loss: 2

tensor(0.7769)

In [81]:


lda = LinearDiscriminantAnalysis()
lda.fit(train_x,train_y)
tran_x = lda.transform(train_x)
trantest_x = lda.transform(test_x)

clf = RFC(min_samples_split=8,bootstrap=False,criterion='entropy',max_depth=20,max_features='sqrt',min_samples_leaf=1,n_estimators=1000)

clf.fit(tran_x,train_y)
print(clf.score(tran_x,train_y))
clf.score(trantest_x,test_y)

1.0


0.71900826446281

In [ ]:
import torch
m = nn.Net(train_data.x.shape[1])
m.load_state_dict(torch.load('model.pth'))


In [ ]:
m(torch.from_numpy(test_x).float()).argmax(1).numpy()

In [71]:
print(clf.best_score_)
print(clf.best_params_)

0.6513184843830006
{'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 8}


In [ ]:
test_data = data.Dataset('test.csv')
test_x = pca.transform(test_data.x)
pred = m(torch.from_numpy(test_x).float()).argmax(1).numpy()
data.write_to_csv('submission.csv',pred,train_data.get_cat_to_label())